In [0]:
# =====================================================
# CONFIGURACIÓN GENERAL
# =====================================================
import os
import requests
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime
from pyspark.sql import Row
from datetime import timedelta


In [0]:
# ======================
# PATHS EN ADLS
# ======================

ALERTS_PATH = "abfss://datos@mastertfm001sta.dfs.core.windows.net/gold/alerts"
STREAMING_CHECKPOINT = "abfss://datos@mastertfm001sta.dfs.core.windows.net/gold/telegram_streaming_checkpoint"

# ======================
# TELEGRAM (DATABRICKS SECRETS)
# ======================

TELEGRAM_TOKEN = "8449014597:AAEB81v89uf43kcND1PZGudO0GQn8coSnnY"
TELEGRAM_CHAT_ID = "-1003724965000"


In [0]:
# dbutils.fs.rm(
#     "abfss://datos@mastertfm001sta.dfs.core.windows.net/gold/telegram_streaming_checkpoint",
#     recurse=True
# )


In [0]:
# =====================================================
# SPARK
# =====================================================

spark = (
    SparkSession.builder
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

In [0]:
# =====================================================
# FUNCIÓN TELEGRAM
# =====================================================

def enviar_telegram(mensaje: str):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"

    if len(mensaje) > 4000:
        mensaje = mensaje[:4000] + "\n…"

    payload = {
        "chat_id": TELEGRAM_CHAT_ID,
        "text": mensaje,
        "parse_mode": "HTML"
    }

    requests.post(url, json=payload, timeout=10)

In [0]:
def format_alert_humana(r):
    ts_exit = r.timestamp_alert
    ts_exit_str = ts_exit.strftime("%Y-%m-%d %H:%M")

    # 🔹 calcular hora de entrada desde hold
    if r.hold_minutes is not None:
        ts_entry = ts_exit - timedelta(minutes=r.hold_minutes)
        ts_entry_str = ts_entry.strftime("%Y-%m-%d %H:%M")
    else:
        ts_entry_str = "N/A"

    sep = "━━━━━━━━━━━━━━"

    if r.signal == "BUY":
        return f"""<pre>
➡️ NUEVA ENTRADA
{sep}
Asset: {r.symbol}
Tipo: {r.tipo_estrategia.capitalize()}

Precio entrada: {r.entry_price}
Hora entrada: {ts_exit_str}
{sep}
Trade ID:
{r.trade_id}
</pre>"""

    # SELL
    pnl = r.pnl_pct
    pnl_emoji = "✅" if pnl > 0 else "❌" if pnl < 0 else "⚠️"
    titulo = (
        "SALIDA CON GANANCIA" if pnl > 0 else
        "SALIDA CON PÉRDIDA" if pnl < 0 else
        "SALIDA NEUTRA"
    )

    horas = r.hold_minutes // 60
    minutos = r.hold_minutes % 60

    return f"""<pre>
{pnl_emoji} {titulo}
{sep}
Asset: {r.symbol}
Tipo: {r.tipo_estrategia.capitalize()}
Motivo: {r.exit_reason}

Precio: {r.entry_price} → {r.price}
Hora entrada: {ts_entry_str}
Hora salida: {ts_exit_str}
{sep}
PnL: {pnl:+.2f}%
Hold: {horas}h {minutos}m
{sep}
Trade ID:
{r.trade_id}
</pre>"""


In [0]:
# def format_alert_humana(r):
#     ts = r.timestamp_alert.strftime("%Y-%m-%d %H:%M")
#     sep = "━━━━━━━━━━━━━━"

#     if r.signal == "BUY":
#         return f"""<pre>
# ➡️ NUEVA ENTRADA
# {sep}
# Asset: {r.symbol}
# Tipo: {r.tipo_estrategia.capitalize()}

# Precio entrada: {r.entry_price}
# Hora: {ts}
# {sep}
# Trade ID:
# {r.trade_id}
# </pre>"""

#     # SELL
#     pnl = r.pnl_pct
#     pnl_emoji = "✅" if pnl > 0 else "❌" if pnl < 0 else "⚠️"
#     titulo = (
#         "SALIDA CON GANANCIA" if pnl > 0 else
#         "SALIDA CON PÉRDIDA" if pnl < 0 else
#         "SALIDA NEUTRA"
#     )

#     horas = r.hold_minutes // 60
#     minutos = r.hold_minutes % 60

#     return f"""<pre>
# {pnl_emoji} {titulo}
# {sep}
# Asset: {r.symbol}
# Tipo: {r.tipo_estrategia.capitalize()}
# Motivo: {r.exit_reason}

# Precio: {r.entry_price} → {r.price}
# Hora: {ts}
# {sep}
# PnL: {pnl:+.2f}%
# Hold: {horas}h {minutos}m
# {sep}
# Trade ID:
# {r.trade_id}
# </pre>"""


In [0]:
# =====================================================
# STREAMING LOGIC
# =====================================================

def process_batch(batch_df, batch_id):
    rows = batch_df.collect()
    if not rows:
        return

    for r in rows:
        msg = format_alert_humana(r)
        enviar_telegram(msg)

In [0]:
# =====================================================
# START STREAM
# =====================================================

alerts_stream = (
    spark.readStream
    .format("delta")
    .option("ignoreDeletes", "true")
    .load(ALERTS_PATH)
)

query = (
    alerts_stream
    .writeStream
    .foreachBatch(process_batch)
    .option("checkpointLocation", STREAMING_CHECKPOINT)
    .outputMode("append")
    .start()
)

query.awaitTermination()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:190)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can